<a href="https://colab.research.google.com/github/swapnilmungi/Patient_Readmission_Prediction/blob/main/PreProcessing_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import joblib
import re

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/HMGT/Data/ImportantColumns.csv')

In [ ]:
df.shape

(96446, 34)

In [ ]:
#V2
df = df.loc[df['readmitted'] != '>30']

In [ ]:
df['readmitted'].value_counts()

NO     50731
<30    11066
Name: readmitted, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61797 entries, 1 to 96445
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0.1              61797 non-null  int64 
 1   Unnamed: 0                61797 non-null  int64 
 2   encounter_id              61797 non-null  int64 
 3   patient_nbr               61797 non-null  int64 
 4   race                      61797 non-null  object
 5   gender                    61797 non-null  object
 6   age                       61797 non-null  object
 7   admission_type_id         61797 non-null  int64 
 8   discharge_disposition_id  61797 non-null  int64 
 9   admission_source_id       61797 non-null  int64 
 10  time_in_hospital          61797 non-null  int64 
 11  num_lab_procedures        61797 non-null  int64 
 12  num_procedures            61797 non-null  int64 
 13  num_medications           61797 non-null  int64 
 14  number_outpatient     

In [ ]:
df['max_glu_serum'].value_counts()

None    58602
Norm     1644
>200      885
>300      666
Name: max_glu_serum, dtype: int64

In [ ]:
df.columns


Index(['Unnamed: 0.1', 'Unnamed: 0', 'encounter_id', 'patient_nbr', 'race',
       'gender', 'age', 'admission_type_id', 'discharge_disposition_id',
       'admission_source_id', 'time_in_hospital', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
       'rosiglitazone', 'insulin', 'metformin-pioglitazone', 'change',
       'diabetesMed', 'readmitted'],
      dtype='object')

In [ ]:
df['A1Cresult'].value_counts()

None    51375
>8       4781
Norm     3240
>7       2401
Name: A1Cresult, dtype: int64

In [ ]:
#Charles Comorbidity Index
def calculateCharles(ICDcode):
  comorbidities = []
  categories = Comorbidities.keys()
  for cat in categories:
    icd9patterns = Comorbidities[cat]
    for icd9pat in icd9patterns:
      pat = re.compile(icd9pat)
      matches = re.match(pat,ICDcode)
      if matches != None: comorbidities.append(cat)
  comorbidities = set(comorbidities)

  weights = [Weights[c] for c in comorbidities]
  #print(weight
  return int(sum(weights))

In [ ]:
#Encoding
df['change'] = df['change'].replace('Ch', 1)
df['change'] = df['change'].replace('No', 0)
df['gender'] = df['gender'].replace('Male', 1)
df['gender'] = df['gender'].replace('Female', 0)
df['diabetesMed'] = df['diabetesMed'].replace('Yes', 1)
df['diabetesMed'] = df['diabetesMed'].replace('No', 0)

#v2 df['readmitted'] = df['readmitted'].replace('>30', 0)
df['readmitted'] = df['readmitted'].replace('<30', 1)
df['readmitted'] = df['readmitted'].replace('NO', 0)

keys = ['metformin',
       'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
       'rosiglitazone', 'insulin', 'metformin-pioglitazone',]

keys_map = [['No']]
for col in keys:
    df[col] = df[col].replace('No', 0)
    df[col] = df[col].replace('Steady', 1)
    df[col] = df[col].replace('Up', 1)
    df[col] = df[col].replace('Down', 1)

In [ ]:
#A1c:

df['A1Cresult'] = df['A1Cresult'].replace('>7', 1)
df['A1Cresult'] = df['A1Cresult'].replace('>8', 2)
df['A1Cresult'] = df['A1Cresult'].replace('Norm', 0)
df['A1Cresult'] = df['A1Cresult'].replace('None', -1)

In [ ]:
#MAx glu serum
df['max_glu_serum'] = df['max_glu_serum'].replace('>200', 1)
df['max_glu_serum'] = df['max_glu_serum'].replace('>300', 2)
df['max_glu_serum'] = df['max_glu_serum'].replace('Norm', 0)
df['max_glu_serum'] = df['max_glu_serum'].replace('None', -1)

In [ ]:
#Diagnosis Variables


In [ ]:
#Age Transformation
def age_transform(age):
  new_age = age[1]+'0'
  return 5+ int(new_age)

df['age'] = df['age'].apply(lambda x : age_transform(x))

In [ ]:
#Race
race_dummies = pd.get_dummies(df.race)
df = pd.concat([df, race_dummies], axis=1)
df.drop('race',axis=1,inplace =True)

# Diagnosis Variables

In [ ]:
Comorbidities = {"MayocardialInfarction": ["410\..*", "412\..*"],
                 "HeartFailure": ["398\.91", "402\.01", "402\.11", "402\.91", "404\.01", "404\.03", "404\.11", "404\.13", "404\.91", "404\.93", "425\..*", "428\..*"],
                 "PeripheralVascularDisease": ["093\.0", "437\.3", "440\..*", "441\..*", "443\..*", "47\.1", "557\.1", "557\.9", "V43\.4"],
                 "CerebrovascularDisease":  ["362\.34", "430\..*", "431\..*", "432\..*", "433\..*", "434\..*", "435\..*", "436\..*", "437\..*", "438\..*"],
                 "Dementia": ["290\..*", "294.1", "331.2"],
                 "ChronicPulmonaryDisease": ["416.8", "416.9", "490\..*", "491\..*", "492\..*", "493\..*", "494\..*", "495\..*", "496\..*", "497\..*", "498\..*", "499\..*", "500\..*", "500\..*", "501\..*", "502\..*", "503\..*", "504\..*", "505\..*", "506.4", "508.1", "508.8"],
                 "RheumaticDisease": ["446.5", "710\..*", "714.0", "714.01", "714.2", "714.8", "725\..*"],
                 "PepticUlcerDisease": ["531\..*", "532\..*", "533\..*", "534\..*"],
                 "MildLiverDisease": ["070.22", "070.23", "070.32", "070.33", "070.44", "070.54", "070.6", "070.9", "570\..*", "571\..*", "573.3", "573.4", "573.8", "573.9", "V42.7"],
                 "DiabetesWithoutComplications": ["250.0", "250.1", "250.2", "250.3", "250.8", "250.9"],
                 "DiabetesWithComplications": ["250.4", "250.5", "250.6", "250.7"],
                 "HemiplegiaParaplegia": ["334.1", "342\..*", "343\..*", "344\..*"],
                 "RenalDisease": ["403.01", "403.11", "403.91", "404.02", "404.03", "404.12", "404.13", "404.92", "404.93", "582\..*", "583\..*", "585\..*", "586\..*", "588.0", "V42.0", "V45.1", "V56\..*"],
                 "Malignancy": ["14\d\..*", "15\d\..*", "16\d\..*", "170\..*", "171\..*", "172\..*", "174\..*", "175\..*", "176\..*", "177\..*", "178\..*", "179\..*", "18\d\..*", "190\..*", "191\..*", "192\..*", "193\..*", "194\..*", "195\..*", "20\d\..*", "238.6"],
                 "ModerateSevereLiverDisease": ["456.0", "456.1", "456.2", "572\..*"],
                 "MetastaticCancer": ["196\..*", "197\..*", "198\..*", "199\..*"],
                 "HIVAIDS": ["042\..*", "043\..*", "044\..*"],
                 "HypertensionUncomplicated": ["401\..*"],
                 "HypertensionComplicated": ["402\..*", "403\..*", "404\..*", "405\..*"],
                 "Coagulopathy":  ["286\..*", "287.1", "287.3", "287.4", "287.5"],
                 "Obesity": ["278.0"],
                 "WeightLoss": ["260\..*", "261\..*", "262\..*", "263\..*", "783.2", "799.4"],
                 "FluidElectrolyteDisorders": ["253.6", "276\..*"]
                }

Weights = {"MayocardialInfarction": 1,
                 "HeartFailure": 1,
                 "PeripheralVascularDisease": 1,
                 "CerebrovascularDisease":  1,
                 "Dementia": 1,
                 "ChronicPulmonaryDisease": 1,
                 "RheumaticDisease": 1,
                 "PepticUlcerDisease": 1,
                 "MildLiverDisease": 1,
                 "DiabetesWithoutComplications": 1,
                 "DiabetesWithComplications": 2,
                 "HemiplegiaParaplegia": 2,
                 "RenalDisease": 2,
                 "Malignancy": 2,
                 "ModerateSevereLiverDisease": 3,
                 "MetastaticCancer": 6,
                 "HIVAIDS": 6,
                 "HypertensionUncomplicated": 0,
                 "HypertensionComplicated": 0,
                 "Coagulopathy": 0,
                 "Obesity": 0,
                 "WeightLoss": 0,
                 "FluidElectrolyteDisorders": 0
                }
def calculateCharles(ICDcode):
  comorbidities = []
  categories = Comorbidities.keys()
  for cat in categories:
    icd9patterns = Comorbidities[cat]
    for icd9pat in icd9patterns:
      pat = re.compile(icd9pat)
      matches = re.match(pat,ICDcode)
      if matches != None: comorbidities.append(cat)
  comorbidities = set(comorbidities)

  weights = [Weights[c] for c in comorbidities]
  return int(sum(weights))

df['Charles_diag_1'] = df['diag_1'].apply(lambda x : calculateCharles(x))
df['Charles_diag_2'] = df['diag_2'].apply(lambda x : calculateCharles(x))
df['Charles_diag_3'] = df['diag_3'].apply(lambda x : calculateCharles(x))

In [ ]:
#dropping diag variables
df.drop(['diag_1','diag_2','diag_3'],axis=1,inplace =True)

In [ ]:
for col in df:
  if df[col].dtype == 'object':
    print(col)
    df[col] = df[col].apply(lambda x : float(x))

In [ ]:
df.head()

,Unnamed: 0.1,Unnamed: 0,encounter_id,patient_nbr,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,diabetesMed,readmitted,AfricanAmerican,Asian,Caucasian,Hispanic,Other,Charles_diag_1,Charles_diag_2,Charles_diag_3
1,1,2,64410,86047875,0,25,1,1,7,2,...,1,0,1,0,0,0,0,0,0,0
2,2,3,500364,82442376,1,35,1,1,7,2,...,1,0,0,0,1,0,0,0,2,0
3,3,4,16680,42519267,1,45,1,1,7,1,...,1,0,0,0,1,0,0,0,0,0
5,5,6,55842,84259809,1,65,3,1,2,4,...,1,0,0,0,1,0,0,0,0,0
7,7,8,12522,48330783,0,85,2,1,4,13,...,1,0,0,0,1,0,0,0,0,0


In [ ]:
#V2
df = df.drop_duplicates(subset= ['patient_nbr'], keep = 'first')

In [ ]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'encounter_id', 'patient_nbr'],axis=1,inplace= True)

In [ ]:
df['readmitted'].value_counts()

0    46741
1     8413
Name: readmitted, dtype: int64

In [ ]:

joblib.dump(df, '/content/drive/MyDrive/HMGT/Data/ModellingReady2classes.pkl')
#df.to_csv('ModellingReady.csv')

['/content/drive/MyDrive/HMGT/Data/ModellingReady2classes.pkl']